# Face Recognition

In [1]:
#pip install opencv-contrib-python

## Steps:
Prepare the training and prediction data with the same way as follows:

  1- Read the images and convert it into grey scales.
  
  2- Face detection to obtain the rectangle of faces or features.
  
  3- Create object of LBPHFaceRecognizer_create. 
  
  4- Use the object of LBPHFaceRecognizer_create to train the model face_recognizer.train()
  
  5- To predict repeate the steps 1,2 to prepare the features of prediction.
  
  6- use the object of LBPHFaceRecognizer_create to train the model to predict face_recognizer.predict()

In [2]:
import cv2 as cv
import numpy as np
import os

In [3]:
# Return the following:
# directory_ls: paths list for all directries for all training data
# label_ls: the names of persons

def obtaining_directries(path):
    directory_ls = []
    label_ls = []
    print(path)
    for root, dirs, files in os.walk(path, topdown=False):
        for name in dirs:
            directory_ls.append(os.path.join(root, name))
            label_ls.append(name)
    return directory_ls, label_ls        


In [18]:
# Return the path for all images in our dataset
def obtaining_image_pathes(directory_ls):
    image_path_ls = []

    # Obtain images' path
    for path in directory_ls:
        for image in os.listdir(path):
            image_path = os.path.join(path,image)
            image_path_ls.append(image_path)
    return image_path_ls        
 
    
# This function is used to detect faces and return them as features.    
def obtaining_features(image_paths):
    features = []
    image_path_ls = []
    image_pixels_ls = []
    #Detect the face inside each image
    haar_cascade = cv.CascadeClassifier('haarcascade_frontalcatface.xml')

    for image_path in image_paths:
        
        # Read images
        image_pixels = cv.imread(image_path)

            
        # Convert to Grey
        gray_pixels = cv.cvtColor(image_pixels, cv.COLOR_BGR2GRAY)
        #cv.imshow('gray_pixels', gray_pixels) 
        #cv.waitKey(0)
        # To detect faces use detectMultiScale()
        haar_rect = haar_cascade.detectMultiScale(gray_pixels, scaleFactor=1.1, minNeighbors=1, minSize=(20, 20))

        for (x,y,w,h) in haar_rect:
            face_matrix = gray_pixels[y:y+h, x:x+w] 
            features.append(face_matrix)
            # this to register the image paths of the images which has been only detected.
            image_path_ls.append(image_path)
            # To add only imges which has been found from face detection
            image_pixels_ls.append(image_pixels)


            
    return features, image_path_ls, image_pixels_ls         

# Obtianing the label by cutting it from the path
def obtaining_labels(image_path_ls, label_ls):
    labels = []
    for image_path in image_path_ls:
        label = image_path.split('\\')[-2]
        labels.append(label_ls.index(label))
    return labels   
    
# This methos is udes to call all above function to prepare features and the true lables in case of training and prediction.
def prepare_training_and_testing(path):
    labels = []
    labels_ls = []
    directory_ls, label_ls = obtaining_directries(path)
    image_paths = obtaining_image_pathes(directory_ls)
    features, image_path_ls, image_pixels_ls = obtaining_features(image_paths)
    labels   = obtaining_labels(image_path_ls, label_ls)

    # Coverting the features and labels into numerical values
    #features = np.array(features, dtype='object')
    return features, labels, label_ls, image_pixels_ls
                
    

In [19]:

def train ():
    training_path = "D:\\Ml_learning\\computer vision\\coursat-ai\\open_cv\\Face_Recognition\\images\\train"
    features, labels, label_ls, image_pixels_ls = prepare_training_and_testing(training_path)
    np.save('features.npy', features)
    np.save('labels.npy', labels)
    print(len(features))
 
    labels = np.array(labels)

    #Create an object for face recognizer
    face_recognizer = cv.face.LBPHFaceRecognizer_create()
    
    #Train the recognizer
    face_recognizer.train(features, labels)
    #save the recognizer
    face_recognizer.save('face_model.yml')
    
    return face_recognizer, label_ls
    


In [20]:
face_recognizer, label_ls = train()

D:\Ml_learning\computer vision\coursat-ai\open_cv\Face_Recognition\images\train
82


In [31]:

# Here we try to predict the image labels.
def predict():
    validation_path = "D:\\Ml_learning\\computer vision\\coursat-ai\\open_cv\\Face_Recognition\\images\\val"
    features, labels, real_labels, image_pixels_ls = prepare_training_and_testing(validation_path)
        
    try:
        for i in range(len(features)):
            #cv.imshow('face recognition', image_pixels_ls[i])
            #cv.waitKey(0)
            if features[i] is not None:
                label, confidence = face_recognizer.predict(features[i])
                print('The real label',str(real_labels[labels[i]]),'\n The expected label is',label, '\n with confidence',confidence, '\n')

            
    except e:
        pass
    

In [32]:
predict()

D:\Ml_learning\computer vision\coursat-ai\open_cv\Face_Recognition\images\val
The real label Erdogan 
 The expected label is 1 
 with confidence 73.60399156766219 

The real label Erdogan 
 The expected label is 4 
 with confidence 141.80304811586012 

The real label Erdogan 
 The expected label is 1 
 with confidence 116.33374781563396 

